In [1]:
import pandas as pd
import numpy as np
import os, glob, sys
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

from sklearn.manifold import TSNE

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

from tqdm import tqdm

device = torch.device('cuda:{}'.format(0) if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device) # change allocation of current GPU 

import warnings
warnings.filterwarnings(action='ignore') 

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

## Get 500 mtorr 

In [2]:
df = pd.read_csv('/home/data/psk/leakage/ID_9.csv')
df.TimeStamp = pd.to_datetime(df.TimeStamp)

col_list = ['TimeStamp','PM1.Gas1_Monitor', 'PM1.Gas2_Monitor', 'PM1.APC_Pressure', 'PM1.APC_Position', 
    'PM1.SourcePwr1_Read', 'PM1.SourcePwr2_Read', 'PM1.Temp1', 'PM1.Temp2',  
    'ApcPosition', 'ApcPositionScaled','WallTemp',
    'Port_Num', 'Process_Num', 'Wafer_Status', 'Folder_Name', 'File_Name', 'Torr','Leak', 'cycle']
df = df.loc[:, col_list]

df_ = df[df.cycle >= 4]
df_.loc[df_.cycle == 4, 'cycle'] = 5

## get cycle info

In [3]:
cycle_num = df_.cycle.unique()
cycle_dict = {}

c_m_list = []
c_s_list = []

for cycle in tqdm(cycle_num):
    df__ = df_[df_.cycle == cycle]
    wafer_unit = list(df__.groupby(['Port_Num', 'Process_Num', 'Wafer_Status', 'Folder_Name', 'File_Name', 'cycle'])) # KEY 값
    wafer_list = list(filter(lambda x: 102 > len(x[1]) > 90, wafer_unit))

    w_mean = np.array([df.iloc[:, 1:-8].mean().values for info, df in wafer_list])
    w_std = np.array([df.iloc[:, 1:-8].std().values for info, df in wafer_list])

    c_mean = np.mean(w_mean, axis = 0)
    c_m_list.append(c_mean)
    
    c_std = np.mean(w_std, axis = 0)
    c_s_list.append(c_std)
    
cycle_dict['mean'] = np.array(c_m_list)
cycle_dict['std'] = np.array(c_s_list)

100%|██████████| 1/1 [00:10<00:00, 10.53s/it]


In [4]:
cycle_dict

{'mean': array([[12756.19298396,  1260.90942482,  1464.4048213 ,   678.18603717,
          2643.8230873 ,  2639.87775034,   248.81512302,   248.66953823,
         33554.38096869,  1470.71486598,    77.60365012]]),
 'std': array([[1.70786988e+03, 2.03284712e+02, 2.12208593e+02, 1.89447918e+02,
         2.16691624e+03, 2.16375136e+03, 5.07406031e-01, 4.86004885e-01,
         7.44315737e+03, 1.87977010e+02, 2.50933011e-01]])}

## Split data

In [5]:
from sklearn.model_selection import train_test_split

wafer_unit = list(df_.groupby(['Port_Num', 'Process_Num', 'Wafer_Status', 'Folder_Name', 'File_Name', 'cycle'])) # KEY 값
wafer_list = list(filter(lambda x: 102 > len(x[1]) > 90, wafer_unit))

X_train_, X_test, y_train, y_test = train_test_split(wafer_list, range(len(wafer_list)), test_size = 0.1, random_state=42)

X_train, X_valid, y_train, y_valid = train_test_split(X_train_, range(len(X_train_)), test_size = 0.1, random_state=42)

split_data = [X_train, X_valid, X_test]

for data in split_data:
    print(pd.Series([wafer[1].Leak.unique()[0] for wafer in data]).value_counts())

0    4213
1     934
dtype: int64
0    478
1     94
dtype: int64
0    523
1    113
dtype: int64


## Dataset

In [6]:
class datasetAE(Dataset):
    def __init__(self, wafer_unit, window_size, max_len, cycle_dict):
        
        super(datasetAE, self).__init__()
        self.wafer_unit = wafer_unit
        self.window_size = window_size
        self.max_len = max_len
        self.cycle_dict = cycle_dict
    
    def __getitem__(self, idx):
        
        info = self.wafer_unit[idx][0]
        df = self.wafer_unit[idx][1]
    
        c_mean = self.cycle_dict['mean'][0]
        c_std = self.cycle_dict['std'][0]
        
        torr = df.Torr.unique()[0]
        y = df.Leak.unique()[0]
        
        df_ = df.iloc[:, 1:-8].reset_index(drop=True)
                
        # padding length
        if len(df_) < self.max_len:
            
            add_num = abs(len(df_) - self.max_len)
            for num in range(add_num):
                df_.loc[len(df_)] = 0
                
        array_df = np.array(df_)
        
        # nomralization with cycle mean and std
        df__ = (array_df - c_mean) / (c_std)
        
        x = np.array(df__).T
        windows_x = np.lib.stride_tricks.sliding_window_view(x, self.window_size, 1)
        
        return np.array( windows_x ), np.array(y), torr
    
    def __len__(self):
        return len(self.wafer_unit)

In [7]:
max_len = max(set([len(wafer[1]) for wafer in wafer_list]))

train_dataset = datasetAE(X_train, 70, max_len, cycle_dict)
valid_dataset = datasetAE(X_valid, 70, max_len, cycle_dict)
test_dataset = datasetAE(X_test, 70, max_len, cycle_dict)

train_loader = DataLoader(train_dataset, batch_size=64, drop_last=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=1)

In [8]:
train_dataset[10][0].shape

(11, 32, 70)

## Model

In [9]:
class MTS_CNN(torch.nn.Module):
    
    # 3 class 모델
    def __init__(self, sensor_num):
        super(MTS_CNN, self).__init__()
        
       
        extract_list = []
        diagnosis_list = []
        for num in range(sensor_num):
            
            extract_layer = torch.nn.Sequential(
                    torch.nn.Conv1d(in_channels = 32, out_channels = 16, 
                                    kernel_size = 5),
                    torch.nn.ReLU(),
                    torch.nn.AvgPool1d(kernel_size = 3, stride = 3),
                    torch.nn.Conv1d(in_channels = 16, out_channels = 64, 
                                    kernel_size = 5),
                    torch.nn.ReLU(),
                    torch.nn.AvgPool1d(kernel_size = 3, stride = 3),
            )

            
            diagnosis_layer = torch.nn.Sequential(
                     torch.nn.Linear(384, 256),
                     torch.nn.ReLU(),
                     torch.nn.Linear(256, 1),
                     torch.nn.ReLU(),
            )
            
            extract_list.append(extract_layer)
            diagnosis_list.append(diagnosis_layer)
        
        self.extract_layer = nn.ModuleList(extract_list)
        self.diagnosis_layer = nn.ModuleList(diagnosis_list)
        
        self.detection_layer = torch.nn.Sequential(
                                             torch.nn.Linear(sensor_num, 32),
                                             torch.nn.ReLU(),
                                             torch.nn.Linear(32, 16),
                                             torch.nn.Dropout(0.5),
                                             torch.nn.ReLU(),
                                             torch.nn.Linear(16, 2)
                                    )

    def forward(self, x):
        b, s, q, w = x.shape
        x_ = x.transpose(1,0)
        
        stack_sensor = []
        # 센서 별로 convolution 적용
        for idx, x__ in enumerate(x_):
            # b, q, w
            # Feature Extraction Layer
            feature = self.extract_layer[idx](x__)
            # (b, 1024)
            flatten = feature.view(b, -1)
            # Fault Diagnosis Layer 
            diagnosis = self.diagnosis_layer[idx](flatten)
            # b, 1
            stack_sensor.append(diagnosis)
        
        # Stack By Sensor
        fin_stack = torch.stack(stack_sensor).transpose(0,1).squeeze()
        
        # Fault Detection Layer
        output = self.detection_layer(fin_stack)
        
        return output, fin_stack

In [10]:
MTS_CNN(11)

MTS_CNN(
  (extract_layer): ModuleList(
    (0): Sequential(
      (0): Conv1d(32, 16, kernel_size=(5,), stride=(1,))
      (1): ReLU()
      (2): AvgPool1d(kernel_size=(3,), stride=(3,), padding=(0,))
      (3): Conv1d(16, 64, kernel_size=(5,), stride=(1,))
      (4): ReLU()
      (5): AvgPool1d(kernel_size=(3,), stride=(3,), padding=(0,))
    )
    (1): Sequential(
      (0): Conv1d(32, 16, kernel_size=(5,), stride=(1,))
      (1): ReLU()
      (2): AvgPool1d(kernel_size=(3,), stride=(3,), padding=(0,))
      (3): Conv1d(16, 64, kernel_size=(5,), stride=(1,))
      (4): ReLU()
      (5): AvgPool1d(kernel_size=(3,), stride=(3,), padding=(0,))
    )
    (2): Sequential(
      (0): Conv1d(32, 16, kernel_size=(5,), stride=(1,))
      (1): ReLU()
      (2): AvgPool1d(kernel_size=(3,), stride=(3,), padding=(0,))
      (3): Conv1d(16, 64, kernel_size=(5,), stride=(1,))
      (4): ReLU()
      (5): AvgPool1d(kernel_size=(3,), stride=(3,), padding=(0,))
    )
    (3): Sequential(
      (0): C

In [11]:
test_tensor = torch.rand(64, 11, 32, 70)
model = MTS_CNN(11)
model(test_tensor)

(tensor([[ 0.0501,  0.0227],
         [-0.0551,  0.0398],
         [-0.0828,  0.1205],
         [ 0.0120,  0.0379],
         [-0.1070,  0.1609],
         [ 0.0019,  0.0605],
         [-0.1071,  0.1212],
         [-0.0833, -0.0645],
         [-0.0291,  0.1114],
         [-0.0817,  0.0844],
         [-0.0078, -0.0416],
         [-0.0092,  0.1128],
         [-0.0168,  0.1289],
         [-0.0749, -0.0192],
         [ 0.0722, -0.0547],
         [-0.0238, -0.0312],
         [-0.0587,  0.0319],
         [ 0.0586,  0.0405],
         [-0.0611,  0.1622],
         [-0.1506, -0.0063],
         [ 0.0086, -0.0371],
         [-0.0109,  0.0339],
         [-0.1194,  0.1180],
         [-0.0757, -0.0212],
         [-0.1223,  0.0488],
         [-0.0059, -0.0046],
         [-0.0952,  0.1045],
         [-0.1203,  0.0219],
         [-0.1268,  0.1253],
         [ 0.0018,  0.0539],
         [-0.0077,  0.0709],
         [-0.0077,  0.1259],
         [-0.0603,  0.0094],
         [-0.2101,  0.0649],
         [-0.1

## Train

In [ ]:
from datetime import datetime

def save_model_by_date(model, optimizer, model_name, save_path, epoch, n_epochs, avg_valid_losses):
    
    global new 
    
    now = datetime.now().strftime('%Y_%m_%d') 
    date_save_path = save_path + '/' + now + '/'
    
    if not os.path.exists(date_save_path):
        os.makedirs(date_save_path)
        new = 1
        
    if epoch == 1:
        new = 0
        if len(glob.glob(date_save_path + '*.pt')) > 0:
            for ex_model in glob.glob(date_save_path + '*.pt'):
                os.remove(ex_model)
            torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, os.path.join(date_save_path, '{}_epoch_{}_valid_loss_{}.pt'.format(model_name, str(epoch), str(avg_valid_losses[-1]))))
        else:
            torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, os.path.join(date_save_path, '{}_epoch_{}_valid_loss_{}.pt'.format(model_name, str(epoch), str(avg_valid_losses[-1]))))
    
    elif new == 1:
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, os.path.join(date_save_path, '{}_epoch_{}_valid_loss_{}.pt'.format(model_name, str(epoch), str(avg_valid_losses[-1]))))
        new = 0
    
    elif n_epochs == epoch:
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, os.path.join(date_save_path, '{}_last_epoch_{}_valid_loss_{}.pt'.format(model_name, str(epoch), str(avg_valid_losses[-1]))))
    
    else:
        if avg_valid_losses[-1] < np.min(avg_valid_losses[:-1]):
            print('updated model saved!')
            ex_model = glob.glob(date_save_path + '*.pt')[0]
            os.remove(ex_model)
            torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, os.path.join(date_save_path, '{}_epoch_{}_valid_loss_{}.pt'.format(model_name, str(epoch), str(avg_valid_losses[-1]))))

In [ ]:
from sklearn.metrics import confusion_matrix

def train_model(model, model_name, save_path, n_epochs, device, train_loader, valid_loader, optimizer, criterion):
    
    model.to(device)
    # to track the average training loss per epoch as the model trains
    avg_train_losses = []
    avg_valid_losses = []
    
    print('start-training')
    for epoch in range(1, n_epochs + 1):
        
        # to track the training loss as the model trains
        train_losses = []
        valid_losses = []
        
        ###################
        # train the model #
        ###################
        model.train() # prep model for training
        for batch, (data, label, torr) in tqdm(enumerate(train_loader, 1)):
            data = data.float().to(device)
            label = label.to(device)
            
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output, fin_stack = model(data)
            # calculate the loss
            loss = criterion(output, label)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
#             print(list(model.layer_dict['0'][0].parameters())[0][0][0])
#             print(list(model.layer_dict['0'][1].parameters())[0][0])
            # record training loss
            train_losses.append(loss.item())

        # print training/validation statistics 
        # calculate average loss over an epoch
        train_loss = np.average(train_losses)
        
        print('epochs : {} / avg_train_loss : {} '.format(epoch, train_loss))
        
        avg_train_losses.append(train_loss)
        
        model.eval()
        with torch.no_grad(): 
            for batch, (data, label, torr) in tqdm(enumerate(valid_loader, 1)):
                data = data.float().to(device)
                label = label.to(device)
                output, fin_stack = model(data)
                loss = criterion(output, label)
                valid_losses.append(loss.item())

            valid_loss = np.average(valid_losses)

            print('epochs : {} / avg_valid_loss : {} '.format(epoch, valid_loss))

            avg_valid_losses.append(valid_loss)
            
        if epoch == 1:
            pass
        else:
            if avg_valid_losses[-1] < np.min(avg_valid_losses[:-1]):
                print('updated model saved!')
                model_info = [epoch, model.state_dict()]
                
        save_model_by_date(model, optimizer, model_name, save_path, epoch, n_epochs, avg_valid_losses)

    return  model, avg_train_losses, avg_valid_losses, model_info

In [ ]:
model = MTS_CNN(11).to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

n_epochs = 200
model_name = 'CLF'
save_path = './Leakage_model/MTS-CNN_Cycle'

model, avg_train_losses, avg_valid_losses, model_info = train_model(model, model_name, save_path, n_epochs, 
                                                                    device, train_loader, valid_loader, optimizer, criterion)

## Test

In [ ]:
model = MTS_CNN(11).to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)
best_model_path = './Leakage_model/MTS-CNN_Cycle/2021_11_17'
model_pt = glob.glob(best_model_path + '/*')[0]
print(model_pt)

checkpoint = torch.load(model_pt)
model.load_state_dict((checkpoint['model_state_dict']))
print('<All keys matched successfully>')

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

def test_model(model, test_dl, device):

    target_list = []
    predict_list = []
    dia_list = []
    
    model.eval()
    with torch.no_grad():
        for num, data in enumerate(tqdm(test_dl)):

            data, labels, torr = data

            data = data.float().to(device)
            labels = labels.to(device)

            outputs, fin_stack= model(data)
            predicted = torch.argmax(outputs.data)
            
            target_list.append(labels.cpu().numpy())
            predict_list.append(predicted.cpu().numpy())

            dia_list.append(fin_stack.cpu())

        fin_target = np.concatenate(target_list, axis = 0)
        fin_pred = np.array(predict_list)
        fin_dia = torch.stack(dia_list)
                
        print(confusion_matrix(fin_target, fin_pred))
        print(classification_report(fin_target, fin_pred))
        
    return fin_target, fin_pred, np.array(fin_dia.view(-1, 11))

In [ ]:
fin_target, fin_pred, fin_dia = test_model(model, test_loader, device)

## Get TSNE graph of fault diagnosis outputs

In [ ]:
from sklearn.manifold import TSNE

embedded = TSNE(n_components=2, learning_rate= 200).fit_transform(fin_dia)
fin_embed = np.concatenate([embedded, fin_target.reshape(-1, 1)] ,axis = 1)
df_embed = pd.DataFrame(fin_embed, columns = ['x', 'y', 'label'])

plt.figure(figsize = [15, 10])
sns.scatterplot(data = df_embed, hue = 'label', x = 'x', y = 'y')
plt.legend()
plt.show()

## Get sensor importance

In [ ]:
for name, param in model.named_parameters():
    if name == 'detection_layer.0.weight':
        weight = param

weights = weight.detach().cpu().numpy()
weights_ = weights.reshape(11, -1)

col_list = ['PM1.Gas1_Monitor', 'PM1.Gas2_Monitor', 'PM1.APC_Pressure', 'PM1.APC_Position', 
    'PM1.SourcePwr1_Read', 'PM1.SourcePwr2_Read', 'PM1.Temp1', 'PM1.Temp2',  
    'ApcPosition', 'ApcPositionScaled','WallTemp']

plt.figure(figsize = [15, 10])
box_list = []
for w in weights_:
    box_list.append(w)
    
plt.boxplot(box_list)
plt.xticks(range(1, 12), col_list,  rotation=20)
plt.show()